### 6개키워드 중 하나만 나오게 하는 샘플모델링

## ./crawled_img/(분위기키워드 폴더명)
 - joyful(활발한)
 - adventure(모험적인)
 - cultural(문화적인)
 - tradition(전통적인)
 - nature(자연)
 - art(예술적인)

### CNN 사용한 샘플모델링(데이터 증강X) - 200장

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# 이미지 크기 설정
img_size = (224, 224)

# 데이터 로드 및 전처리 함수
def load_and_preprocess_data(image_paths, label, brightness=True):
    data = []
    labels = []

    for img_path in image_paths:
        img = load_img(img_path, target_size=img_size)
        img_array = img_to_array(img)

        # 이미지 명도와 채도 높이기
        if brightness:
            img_array = cv2.convertScaleAbs(img_array, alpha=1.2, beta=30)

        img_array = img_array.astype('float32') / 255.0  # 이미지를 0과 1 사이 값으로 정규화

        data.append(img_array)
        labels.append(label)

    return np.array(data), np.array(labels)

# 카테고리 리스트
categories = ["joyful", "adventure", "tradition", "nature", "cultural", "art"]
num_classes = len(categories)

# 이미지 데이터 로드 및 전처리
all_data = []
all_labels = []

for i, category in enumerate(categories):
    category_path = os.path.join("crawled_img", category)
    category_images = [os.path.join(category_path, img) for img in os.listdir(category_path)[:200]]  # 처음 200개 이미지만 선택

    data, labels = load_and_preprocess_data(category_images, label=i)
    all_data.extend(data)
    all_labels.extend(labels)

# 데이터 합치기 및 레이블 변환
X = np.array(all_data)
y = to_categorical(np.array(all_labels), num_classes=num_classes)

# 학습 및 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN 모델 생성
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
30/30 [==============================] - 26s 828ms/step - loss: 1.7257 - accuracy: 0.2760 - val_loss: 1.6282 - val_accuracy: 0.3417
Epoch 2/20
30/30 [==============================] - 26s 867ms/step - loss: 1.3796 - accuracy: 0.4396 - val_loss: 1.2716 - val_accuracy: 0.4750
Epoch 3/20
30/30 [==============================] - 28s 953ms/step - loss: 1.0722 - accuracy: 0.5865 - val_loss: 1.2062 - val_accuracy: 0.5167
Epoch 4/20
30/30 [==============================] - 34s 1s/step - loss: 0.8515 - accuracy: 0.6792 - val_loss: 1.3690 - val_accuracy: 0.5000
Epoch 5/20
30/30 [==============================] - 47s 2s/step - loss: 0.5925 - accuracy: 0.7990 - val_loss: 1.3658 - val_accuracy: 0.4833
Epoch 6/20
30/30 [==============================] - 50s 2s/step - loss: 0.3522 - accuracy: 0.8771 - val_loss: 1.3723 - val_accuracy: 0.5792
Epoch 7/20
30/30 [==============================] - 50s 2s/step - loss: 0.1655 - accuracy: 0.9479 - val_loss: 1.2929 - val_accuracy: 0.6083
Epoch 8/20


#### 결과
- **훈련 데이터는 높은 정확도를 보임, 하지만 검증 데이터는 정확도가 낮은 양상을 보임**
- **즉, 과적합이 발생**
    - 모델이 훈련데이터에 맞춰져있어 새로운 데이터를 일반화시키지 못함
- **epoch = 20, batch_size = 32, test_size=0.2**
- **코드 수정 방안**
    - 데이터 증강
        - 증강을 너무 많이 쓰면 훈련속도 느려지고, 과도한 증강추가는 효과적이지 않을수있음
    - 모델 복잡도 조정
    - dropout
    - 하이퍼파라미터 조정
    - 에포크 수 조정
    - 더 많은 데이터로 학습시키기
    - 다양한 최적화 알고리즘 사용 : SGD, RMSprop
    - 데이터 제너레이터

## _______________________________________________________

### CNN 사용한 샘플모델링(데이터 증강) - 이미지 200장

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# 이미지 크기 설정
img_size = (224, 224)
max_images_per_category = 200

# 데이터 로드 및 전처리 함수
def load_and_preprocess_data(image_paths, label, brightness=True):
    data = []
    labels = []

    # 처음부터 200장만 선택
    selected_images = image_paths[:max_images_per_category]

    for img_path in selected_images:
        img = cv2.imread(img_path)

        # 이미지가 비어 있는지 확인
        if img is None:
            print(f"이미지 로딩 중 오류 발생: {img_path}")
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, img_size)

        # 이미지 명도와 채도 높이기
        if brightness:
            img = cv2.convertScaleAbs(img, alpha=1.2, beta=30)

        img_array = img.astype('float32') / 255.0  # 이미지를 0과 1 사이 값으로 정규화

        data.append(img_array)
        labels.append(label)

    return np.array(data), np.array(labels)

# 카테고리 리스트
categories = ["joyful", "adventure", "tradition", "nature", "cultural", "art"]
num_classes = len(categories)

# 데이터 로드 및 전처리 함수를 이용하여 데이터 읽기
all_data = []
all_labels = []

for i, category in enumerate(categories):
    category_path = os.path.join("crawled_img", category)
    category_images = [os.path.join(category_path, img) for img in os.listdir(category_path)]

    data, labels = load_and_preprocess_data(category_images, label=i)
    all_data.extend(data)
    all_labels.extend(labels)

# 데이터 합치기 및 레이블 변환
X = np.array(all_data)
y = to_categorical(np.array(all_labels), num_classes=num_classes)

# 학습 및 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN 모델 생성
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 데이터 증강을 적용하여 모델 훈련
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)
model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=20, validation_data=(X_test, y_test))


Epoch 1/20
30/30 [==============================] - 30s 944ms/step - loss: 2.0864 - accuracy: 0.1677 - val_loss: 1.7518 - val_accuracy: 0.1958
Epoch 2/20
30/30 [==============================] - 30s 997ms/step - loss: 1.7463 - accuracy: 0.2469 - val_loss: 1.6693 - val_accuracy: 0.2875
Epoch 3/20
30/30 [==============================] - 40s 1s/step - loss: 1.6676 - accuracy: 0.2958 - val_loss: 1.5040 - val_accuracy: 0.3542
Epoch 4/20
30/30 [==============================] - 48s 2s/step - loss: 1.5474 - accuracy: 0.3552 - val_loss: 2.0581 - val_accuracy: 0.2417
Epoch 5/20
30/30 [==============================] - 52s 2s/step - loss: 1.4991 - accuracy: 0.3531 - val_loss: 1.6579 - val_accuracy: 0.3625
Epoch 6/20
30/30 [==============================] - 52s 2s/step - loss: 1.4011 - accuracy: 0.4094 - val_loss: 1.9326 - val_accuracy: 0.2292
Epoch 7/20
30/30 [==============================] - 51s 2s/step - loss: 1.3927 - accuracy: 0.4313 - val_loss: 1.5280 - val_accuracy: 0.4083
Epoch 8/20
30/

### CNN 사용한 샘플모델링(데이터 증강, 알고리즘 sgd) - 이미지 200장

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# 이미지 크기 설정
img_size = (224, 224)
max_images_per_category = 200

# 데이터 로드 및 전처리 함수
def load_and_preprocess_data(image_paths, label, brightness=True):
    data = []
    labels = []

    # 처음부터 200장만 선택
    selected_images = image_paths[:max_images_per_category]

    for img_path in selected_images:
        img = cv2.imread(img_path)

        # 이미지가 비어 있는지 확인
        if img is None:
            print(f"이미지 로딩 중 오류 발생: {img_path}")
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, img_size)

        # 이미지 명도와 채도 높이기
        if brightness:
            img = cv2.convertScaleAbs(img, alpha=1.2, beta=30)

        img_array = img.astype('float32') / 255.0  # 이미지를 0과 1 사이 값으로 정규화

        data.append(img_array)
        labels.append(label)

    return np.array(data), np.array(labels)

# 카테고리 리스트
categories = ["joyful", "adventure", "tradition", "nature", "cultural", "art"]
num_classes = len(categories)

# 데이터 로드 및 전처리 함수를 이용하여 데이터 읽기
all_data = []
all_labels = []

for i, category in enumerate(categories):
    category_path = os.path.join("crawled_img", category)
    category_images = [os.path.join(category_path, img) for img in os.listdir(category_path)]

    data, labels = load_and_preprocess_data(category_images, label=i)
    all_data.extend(data)
    all_labels.extend(labels)

# 데이터 합치기 및 레이블 변환
X = np.array(all_data)
y = to_categorical(np.array(all_labels), num_classes=num_classes)

# 학습 및 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN 모델 생성
model_sgd = Sequential()
model_sgd.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model_sgd.add(MaxPooling2D((2, 2)))
model_sgd.add(Conv2D(64, (3, 3), activation='relu'))
model_sgd.add(MaxPooling2D((2, 2)))
model_sgd.add(Conv2D(128, (3, 3), activation='relu'))
model_sgd.add(MaxPooling2D((2, 2)))
model_sgd.add(Flatten())
model_sgd.add(Dense(128, activation='relu'))
model_sgd.add(Dense(num_classes, activation='softmax'))

# 모델 컴파일 - 'sgd' 옵티마이저
model_sgd.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# 데이터 증강을 적용하여 모델 훈련 - 'sgd' 모델
datagen_sgd = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen_sgd.fit(X_train)
model_sgd.fit(datagen_sgd.flow(X_train, y_train, batch_size=32), epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
30/30 [==============================] - 30s 956ms/step - loss: 1.7923 - accuracy: 0.1833 - val_loss: 1.7694 - val_accuracy: 0.1708
Epoch 2/20
30/30 [==============================] - 30s 987ms/step - loss: 1.7682 - accuracy: 0.2469 - val_loss: 1.7356 - val_accuracy: 0.1875
Epoch 3/20
30/30 [==============================] - 31s 1s/step - loss: 1.7237 - accuracy: 0.2469 - val_loss: 1.6622 - val_accuracy: 0.3792
Epoch 4/20
30/30 [==============================] - 31s 1s/step - loss: 1.6860 - accuracy: 0.2937 - val_loss: 1.5926 - val_accuracy: 0.3500
Epoch 5/20
30/30 [==============================] - 42s 1s/step - loss: 1.6818 - accuracy: 0.2854 - val_loss: 1.6450 - val_accuracy: 0.2292
Epoch 6/20
30/30 [==============================] - 53s 2s/step - loss: 1.6946 - accuracy: 0.2865 - val_loss: 1.6835 - val_accuracy: 0.2375
Epoch 7/20
30/30 [==============================] - 60s 2s/step - loss: 1.6482 - accuracy: 0.3167 - val_loss: 1.5887 - val_accuracy: 0.3542
Epoch 8/20
30/

### CNN 사용한 샘플모델링(데이터 증강, 알고리즘 rmsprop) - 이미지 200장

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# 이미지 크기 설정
img_size = (224, 224)
max_images_per_category = 200

# 데이터 로드 및 전처리 함수
def load_and_preprocess_data(image_paths, label, brightness=True):
    data = []
    labels = []

    # 처음부터 200장만 선택
    selected_images = image_paths[:max_images_per_category]

    for img_path in selected_images:
        img = cv2.imread(img_path)

        # 이미지가 비어 있는지 확인
        if img is None:
            print(f"이미지 로딩 중 오류 발생: {img_path}")
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, img_size)

        # 이미지 명도와 채도 높이기
        if brightness:
            img = cv2.convertScaleAbs(img, alpha=1.2, beta=30)

        img_array = img.astype('float32') / 255.0  # 이미지를 0과 1 사이 값으로 정규화

        data.append(img_array)
        labels.append(label)

    return np.array(data), np.array(labels)

# 카테고리 리스트
categories = ["joyful", "adventure", "tradition", "nature", "cultural", "art"]
num_classes = len(categories)

# 데이터 로드 및 전처리 함수를 이용하여 데이터 읽기
all_data = []
all_labels = []

for i, category in enumerate(categories):
    category_path = os.path.join("crawled_img", category)
    category_images = [os.path.join(category_path, img) for img in os.listdir(category_path)]

    data, labels = load_and_preprocess_data(category_images, label=i)
    all_data.extend(data)
    all_labels.extend(labels)

# 데이터 합치기 및 레이블 변환
X = np.array(all_data)
y = to_categorical(np.array(all_labels), num_classes=num_classes)

# 학습 및 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN 모델 생성 - 'rmsprop' 옵티마이저
model_rmsprop = Sequential()
model_rmsprop.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model_rmsprop.add(MaxPooling2D((2, 2)))
model_rmsprop.add(Conv2D(64, (3, 3), activation='relu'))
model_rmsprop.add(MaxPooling2D((2, 2)))
model_rmsprop.add(Conv2D(128, (3, 3), activation='relu'))
model_rmsprop.add(MaxPooling2D((2, 2)))
model_rmsprop.add(Flatten())
model_rmsprop.add(Dense(128, activation='relu'))
model_rmsprop.add(Dense(num_classes, activation='softmax'))

# 모델 컴파일 - 'rmsprop' 옵티마이저
model_rmsprop.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# 데이터 증강을 적용하여 모델 훈련 - 'rmsprop' 모델
datagen_rmsprop = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen_rmsprop.fit(X_train)
model_rmsprop.fit(datagen_rmsprop.flow(X_train, y_train, batch_size=32), epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
30/30 [==============================] - 43s 1s/step - loss: 3.6052 - accuracy: 0.1771 - val_loss: 1.7331 - val_accuracy: 0.2125
Epoch 2/20
30/30 [==============================] - 60s 2s/step - loss: 1.8132 - accuracy: 0.2458 - val_loss: 1.6640 - val_accuracy: 0.2333
Epoch 3/20
30/30 [==============================] - 65s 2s/step - loss: 1.6907 - accuracy: 0.2917 - val_loss: 1.4815 - val_accuracy: 0.4042
Epoch 4/20
30/30 [==============================] - 45s 1s/step - loss: 1.5725 - accuracy: 0.3365 - val_loss: 1.5665 - val_accuracy: 0.3292
Epoch 5/20
30/30 [==============================] - 56s 2s/step - loss: 1.6032 - accuracy: 0.3562 - val_loss: 1.3667 - val_accuracy: 0.4875
Epoch 6/20
30/30 [==============================] - 45s 1s/step - loss: 1.5710 - accuracy: 0.3948 - val_loss: 1.3209 - val_accuracy: 0.4750
Epoch 7/20
30/30 [==============================] - 53s 2s/step - loss: 1.4105 - accuracy: 0.4229 - val_loss: 1.2877 - val_accuracy: 0.4500
Epoch 8/20
30/30 [==

## 데이터 증강 이미지 200 에포크 40

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# 이미지 크기 설정
img_size = (224, 224)
max_images_per_category = 200

# 데이터 로드 및 전처리 함수
def load_and_preprocess_data(image_paths, label, brightness=True):
    data = []
    labels = []

    # 처음부터 200장만 선택
    selected_images = image_paths[:max_images_per_category]

    for img_path in selected_images:
        img = cv2.imread(img_path)

        # 이미지가 비어 있는지 확인
        if img is None:
            print(f"이미지 로딩 중 오류 발생: {img_path}")
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, img_size)

        # 이미지 명도와 채도 높이기
        if brightness:
            img = cv2.convertScaleAbs(img, alpha=1.2, beta=30)

        img_array = img.astype('float32') / 255.0  # 이미지를 0과 1 사이 값으로 정규화

        data.append(img_array)
        labels.append(label)

    return np.array(data), np.array(labels)

# 카테고리 리스트
categories = ["joyful", "adventure", "tradition", "nature", "cultural", "art"]
num_classes = len(categories)

# 데이터 로드 및 전처리 함수를 이용하여 데이터 읽기
all_data = []
all_labels = []

for i, category in enumerate(categories):
    category_path = os.path.join("crawled_img", category)
    category_images = [os.path.join(category_path, img) for img in os.listdir(category_path)]

    data, labels = load_and_preprocess_data(category_images, label=i)
    all_data.extend(data)
    all_labels.extend(labels)

# 데이터 합치기 및 레이블 변환
X = np.array(all_data)
y = to_categorical(np.array(all_labels), num_classes=num_classes)

# 학습 및 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN 모델 생성
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 데이터 증강을 적용하여 모델 훈련
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)
model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=40, validation_data=(X_test, y_test))


Epoch 1/40
30/30 [==============================] - 79s 3s/step - loss: 2.5464 - accuracy: 0.1708 - val_loss: 1.6597 - val_accuracy: 0.2917
Epoch 2/40
30/30 [==============================] - 60s 2s/step - loss: 1.6117 - accuracy: 0.3073 - val_loss: 1.8939 - val_accuracy: 0.2625
Epoch 3/40
30/30 [==============================] - 60s 2s/step - loss: 1.5272 - accuracy: 0.3531 - val_loss: 1.4012 - val_accuracy: 0.3750
Epoch 4/40
30/30 [==============================] - 54s 2s/step - loss: 1.4305 - accuracy: 0.3906 - val_loss: 1.9176 - val_accuracy: 0.3292
Epoch 5/40
30/30 [==============================] - 55s 2s/step - loss: 1.3523 - accuracy: 0.4500 - val_loss: 1.8667 - val_accuracy: 0.3375
Epoch 6/40
30/30 [==============================] - 58s 2s/step - loss: 1.3364 - accuracy: 0.4750 - val_loss: 1.2319 - val_accuracy: 0.5250
Epoch 7/40
30/30 [==============================] - 52s 2s/step - loss: 1.2652 - accuracy: 0.5010 - val_loss: 1.2202 - val_accuracy: 0.5167
Epoch 8/40
30/30 [==

In [ ]:
# 모델 평가
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"테스트 손실: {test_loss}")
print(f"테스트 정확도: {test_accuracy}")

8/8 [==============================] - 2s 175ms/step - loss: 1.4169 - accuracy: 0.5708
테스트 손실: 1.4168516397476196
테스트 정확도: 0.5708333253860474


### 데이터 증강 이미지 200 에포크 40 얼리스타핑

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping

# 이미지 크기 설정
img_size = (224, 224)
max_images_per_category = 200

# 데이터 로드 및 전처리 함수
def load_and_preprocess_data(image_paths, label, brightness=True):
    data = []
    labels = []

    # 처음부터 200장만 선택
    selected_images = image_paths[:max_images_per_category]

    for img_path in selected_images:
        img = cv2.imread(img_path)

        # 이미지가 비어 있는지 확인
        if img is None:
            print(f"이미지 로딩 중 오류 발생: {img_path}")
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, img_size)

        # 이미지 명도와 채도 높이기
        if brightness:
            img = cv2.convertScaleAbs(img, alpha=1.2, beta=30)

        img_array = img.astype('float32') / 255.0  # 이미지를 0과 1 사이 값으로 정규화

        data.append(img_array)
        labels.append(label)

    return np.array(data), np.array(labels)

# 카테고리 리스트
categories = ["joyful", "adventure", "tradition", "nature", "cultural", "art"]
num_classes = len(categories)

# 데이터 로드 및 전처리 함수를 이용하여 데이터 읽기
all_data = []
all_labels = []

for i, category in enumerate(categories):
    category_path = os.path.join("crawled_img", category)
    category_images = [os.path.join(category_path, img) for img in os.listdir(category_path)]

    data, labels = load_and_preprocess_data(category_images, label=i)
    all_data.extend(data)
    all_labels.extend(labels)

# 데이터 합치기 및 레이블 변환
X = np.array(all_data)
y = to_categorical(np.array(all_labels), num_classes=num_classes)

# 학습 및 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN 모델 생성
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(monitor='val_accuracy',  # 모니터링할 지표
                               patience=15,             # 지정된 epochs 동안 개선이 없을 경우 학습 중지
                               restore_best_weights=True)  # 가장 좋은 가중치로 복원

# 데이터 증강을 적용하여 모델 훈련
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    epochs=40,
                    validation_data=(X_test, y_test),
                    callbacks=[early_stopping])


Epoch 1/40
30/30 [==============================] - 29s 935ms/step - loss: 1.8752 - accuracy: 0.2604 - val_loss: 1.6074 - val_accuracy: 0.3417
Epoch 2/40
30/30 [==============================] - 30s 992ms/step - loss: 1.5372 - accuracy: 0.3375 - val_loss: 1.6017 - val_accuracy: 0.3625
Epoch 3/40
30/30 [==============================] - 31s 1s/step - loss: 1.4657 - accuracy: 0.4031 - val_loss: 1.8217 - val_accuracy: 0.2667
Epoch 4/40
30/30 [==============================] - 31s 1s/step - loss: 1.3872 - accuracy: 0.4417 - val_loss: 1.3241 - val_accuracy: 0.5000
Epoch 5/40
30/30 [==============================] - 34s 1s/step - loss: 1.3616 - accuracy: 0.4479 - val_loss: 1.3528 - val_accuracy: 0.4542
Epoch 6/40
30/30 [==============================] - 53s 2s/step - loss: 1.3030 - accuracy: 0.4760 - val_loss: 1.3088 - val_accuracy: 0.4750
Epoch 7/40
30/30 [==============================] - 55s 2s/step - loss: 1.2098 - accuracy: 0.5354 - val_loss: 1.0663 - val_accuracy: 0.5708
Epoch 8/40
30/

In [ ]:
# 테스트 데이터에서 모델 평가
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

8/8 [==============================] - 1s 155ms/step - loss: 0.8578 - accuracy: 0.6750
Test Accuracy: 0.6750
Test Loss: 0.8578


In [ ]:
from tensorflow.keras.models import save_model, load_model

# 전체 모델 저장
model.save("keyword_image.h5")

# 전체 모델 불러오기
loaded_model = load_model("keyword_image.h5")

### 데이터 증강, 이미지 200, 에포크 60

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping

# 이미지 크기 설정
img_size = (224, 224)
max_images_per_category = 200

# 데이터 로드 및 전처리 함수
def load_and_preprocess_data(image_paths, label, brightness=True):
    data = []
    labels = []

    # 처음부터 200장만 선택
    selected_images = image_paths[:max_images_per_category]

    for img_path in selected_images:
        img = cv2.imread(img_path)

        # 이미지가 비어 있는지 확인
        if img is None:
            print(f"이미지 로딩 중 오류 발생: {img_path}")
            continue

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, img_size)

        # 이미지 명도와 채도 높이기
        if brightness:
            img = cv2.convertScaleAbs(img, alpha=1.2, beta=30)

        img_array = img.astype('float32') / 255.0  # 이미지를 0과 1 사이 값으로 정규화

        data.append(img_array)
        labels.append(label)

    return np.array(data), np.array(labels)

# 카테고리 리스트
categories = ["joyful", "adventure", "tradition", "nature", "cultural", "art"]
num_classes = len(categories)

# 데이터 로드 및 전처리 함수를 이용하여 데이터 읽기
all_data = []
all_labels = []

for i, category in enumerate(categories):
    category_path = os.path.join("crawled_img", category)
    category_images = [os.path.join(category_path, img) for img in os.listdir(category_path)]

    data, labels = load_and_preprocess_data(category_images, label=i)
    all_data.extend(data)
    all_labels.extend(labels)

# 데이터 합치기 및 레이블 변환
X = np.array(all_data)
y = to_categorical(np.array(all_labels), num_classes=num_classes)

# 학습 및 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN 모델 생성
model1 = Sequential()
model1.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model1.add(MaxPooling2D((2, 2)))
model1.add(Conv2D(64, (3, 3), activation='relu'))
model1.add(MaxPooling2D((2, 2)))
model1.add(Conv2D(128, (3, 3), activation='relu'))
model1.add(MaxPooling2D((2, 2)))
model1.add(Flatten())
model1.add(Dense(128, activation='relu'))
model1.add(Dense(num_classes, activation='softmax'))

# 모델 컴파일
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(monitor='val_accuracy',  # 모니터링할 지표
                               patience=15,             # 지정된 epochs 동안 개선이 없을 경우 학습 중지
                               restore_best_weights=True)  # 가장 좋은 가중치로 복원

# 데이터 증강을 적용하여 모델 훈련
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)
history = model1.fit(datagen.flow(X_train, y_train, batch_size=32),
                    epochs=60,
                    validation_data=(X_test, y_test),
                    callbacks=[early_stopping])


Epoch 1/60
30/30 [==============================] - 29s 906ms/step - loss: 2.0089 - accuracy: 0.2115 - val_loss: 1.6545 - val_accuracy: 0.3042
Epoch 2/60
30/30 [==============================] - 30s 982ms/step - loss: 1.5587 - accuracy: 0.3396 - val_loss: 1.9513 - val_accuracy: 0.2667
Epoch 3/60
30/30 [==============================] - 30s 1s/step - loss: 1.4897 - accuracy: 0.3656 - val_loss: 1.6041 - val_accuracy: 0.3750
Epoch 4/60
30/30 [==============================] - 31s 1s/step - loss: 1.4408 - accuracy: 0.4208 - val_loss: 1.8316 - val_accuracy: 0.3333
Epoch 5/60
30/30 [==============================] - 34s 1s/step - loss: 1.3689 - accuracy: 0.4583 - val_loss: 1.7818 - val_accuracy: 0.3333
Epoch 6/60
30/30 [==============================] - 50s 2s/step - loss: 1.3081 - accuracy: 0.4865 - val_loss: 1.3914 - val_accuracy: 0.4333
Epoch 7/60
30/30 [==============================] - 64s 2s/step - loss: 1.2542 - accuracy: 0.5031 - val_loss: 1.5296 - val_accuracy: 0.4125
Epoch 8/60
30/

- epoch 수를 늘리수록 train_loss랑 train_accuracy는 수치가 좋아짐
- val_loss는 평균 0.5~0.6 수치를 보임